## Spanish Distribution Grid

In [ ]:
import json, math, inspect
import numpy as np
import pandas as pd
import pandapower as pp
from tqdm.notebook import trange, tqdm
from pathlib import Path
from opendssdirect import dss

grid_path = Path.cwd() / "data" / "SpanishLVNetwork" / "RunDss"
print('OpenDSSDirect.py and engine versions:', dss.Version())
dss(f"Redirect '{str(grid_path / 'Master.dss')}'")

### Buses

In [ ]:
net = pp.create_empty_network(f_hz=50.0)

dss.Command("MakeBusList") # Force instantiation of all buses
dss.Command("CalcVoltageBases") # Auto Set Voltage Bases
for i, bus_name in tqdm(enumerate(dss.Circuit.AllBusNames()), total=dss.Circuit.NumBuses()):
    # if "_" not in bus_name: # 10289 -> 2793
    dss.Circuit.SetActiveBus(bus_name)
    kV = dss.Bus.kVBase()
    X, Y = dss.Bus.X(), dss.Bus.Y()
    pp.create_bus(net, kV, name=bus_name, geodata=(X,Y), type="b")
net.bus_geodata.loc[0, ["x","y"]] = (net.bus_geodata.x.max(), net.bus_geodata.y.max())
# pp.create_ext_grid(net, name="VSource", bus=0, vm_pu=1, va_degree=0.0)

### Lines and Switches
Also creates infinite Voltage Sources (external grids) at each MV connection point.

In [ ]:
from dss import LineUnits as LU

bus_0 = net.bus.loc[0]
unit_conversion = {LU.km:1, LU.meter:1000, LU.cm:100000, LU.mm:1000000, LU.Miles:1.609344, LU.ft:3280.84, LU.inch:39370.08, LU.kFt:3.280839895}
ext_grids = []
for i, line in tqdm(enumerate(dss.Lines), total=len(dss.Lines.AllNames())):
    name = line.Name()
    idx = line.Idx()

    # Deduce which buses this line is connected to
    bus1_name, *bus1_terminals = line.Bus1().split(".")
    bus1_id = net.bus.index[net.bus.name == bus1_name][0]
    bus2_name, *bus2_terminals = line.Bus2().split(".")
    bus2_id =  net.bus.index[net.bus.name == bus2_name][0]
    # Line / Switch already exists (10258 -> 2443)
    # if bus1_id == bus2_id:
    #     continue
    # elif np.logical_and(net.line.from_bus == bus1_id, net.line.to_bus == bus2_id).any():
    #     continue
    # Deduce length in km
    length = line.Length()
    dss.Circuit.SetActiveClass("LineCode")
    dss.Circuit.SetActiveElement(line.LineCode())
    length_unit = dss.LineCodes.Units()
    length_km = length * unit_conversion[length_unit]

    # Retrieve Line Properties
    C0, C1 = line.C0(), line.C1() # nF / unit length
    R0, R1 = line.R0(), line.R1() # Ohms / unit length
    X0, X1 = line.X0(), line.X1() # Ohms / unit length
    n_phases = line.Phases()
    normAmps, extAmps = line.NormAmps(), line.EmergAmps() # Note: We assume Normal Amps is the thermal limit

    is_switch = line.IsSwitch()
    
    # Find Location / Coordinates of Line Ends
    x1, y1 = net.bus_geodata.loc[bus1_id, ["x","y"]]
    x2, y2 = net.bus_geodata.loc[bus2_id, ["x","y"]]
    if bus1_id == 0 and bus2_id != 0:
        new_mv_bus_name = f"Source{name.strip('mv')}"
        x1, y1 = x2, y2
        if new_mv_bus_name in net.bus.name:
            from_bus = net.bus.index[net.bus.name == new_mv_bus_name]
        else:
            from_bus = pp.create_bus(net, bus_0.vn_kv, name=new_mv_bus_name, geodata=((x1,y1)), type="b")
            # Create Infinite Voltage Source (to represent MV grid)
            pp.create_ext_grid(net, from_bus, name=f"VSource{name.strip('mv')}", vm_pu=1.0, va_degree=0)
    else:
        from_bus = bus1_id
    if bus2_id == 0 and bus1_id != 0:
        new_mv_bus_name = f"Source{name.strip('mv')}"
        x2, y2 = x1, y1
        if new_mv_bus_name in net.bus.name:
            to_bus = net.bus.index[net.bus.name == new_mv_bus_name]
        else:
            to_bus = pp.create_bus(net, bus_0.vn_kv, name=f"Source{name.strip('mv')}", geodata=((x2,y2)), type="b")
            # Create Infinite Voltage Source (to represent MV grid)
            pp.create_ext_grid(net, to_bus, name=f"VSource{name.strip('mv')}", vm_pu=1.0, va_degree=0)
    else: # Connection to MV Network
        to_bus = bus2_id
    geodata = ((x1,y1), (x2,y2))
    
    
    if "fuse" in name: # Create Bus-Bus Switch
        pp.create_switch(net, name=name, bus=from_bus, element=to_bus, et="b", closed=True,
                        type="CB", in_ka=extAmps, z_ohm=2.5e-5)
    elif "cktbk" in name:
        pp.create_switch(net, name=name, bus=from_bus, element=to_bus, et="b", closed=True,
                        type="CB", in_ka=extAmps, z_ohm=math.sqrt(math.pow(R0,2)+math.pow(X0, 2)))
    else: # Create Line in PandaPower
        pp.create_line_from_parameters(net, name=name, type="ol", 
                                    from_bus=from_bus, to_bus=to_bus,
                                    length_km=length_km,
                                    r_ohm_per_km=R1, x_ohm_per_km=X1, c_nf_per_km=C1, max_i_ka=normAmps/1000.0, 
                                    geodata=geodata, in_service=True, parallel=n_phases, 
                                    max_loading_percent=100.0,
                                    r0_ohm_per_km=R0, x0_ohm_per_km=X0, c0_nf_per_km=C0)

In [ ]:
line_segments = {}
for i, line_idx in enumerate(net.line.index):
    line_name = net.line.loc[line_idx, "name"]
    main_bus, *_ = line_name.split("_")
    line_segments[main_bus] = [line_idx] if main_bus not in line_segments else line_segments[main_bus] + [line_idx]

In [ ]:
import copy
import warnings
drop_count = 0
buses_to_drop = []
for main_bus, segments in line_segments.items():
    if len(segments) > 1:
        first_seg_idx = segments[0]
        line_properties = net.line.loc[first_seg_idx, :].to_dict()
        [line_properties.pop(elt) for elt in ["name", "std_type", "from_bus", "to_bus", "length_km"]]
        total_length = net.line.loc[segments, "length_km"].sum()
        first_name = net.line.loc[first_seg_idx, "name"]
        from_bus = net.line.loc[first_seg_idx, "from_bus"]
        from_bus_name = net.bus.loc[from_bus, "name"]
        last_seg_idx = segments[-1]
        last_name = net.line.loc[last_seg_idx, "name"]
        to_bus = net.line.loc[last_seg_idx, "to_bus"]
        to_bus_name = net.bus.loc[to_bus, "name"]
        pp.create_line_from_parameters(
            net, name=last_name, from_bus=from_bus, to_bus=to_bus, length_km=total_length,
            geodata=(net.line_geodata.loc[first_seg_idx, "coords"][0], net.line_geodata.loc[last_seg_idx, "coords"][-1]),
            **line_properties)
        with warnings.catch_warnings(action="ignore"):
            buses = [net.line.loc[first_seg_idx, "to_bus"]] + list(net.line.loc[segments[1:-1], ["from_bus", "to_bus"]].to_numpy().flatten()) + [net.line.loc[last_seg_idx, "from_bus"]]
            buses_to_drop.extend(buses)
            pp.drop_lines(net, pd.Series(segments))
        drop_count += len(segments)
buses_to_drop = np.unique(buses_to_drop)
pp.drop_buses(net, buses_to_drop, drop_elements=False)
print(f"Dropped {drop_count} lines")
print(f"Droppped {len(buses_to_drop)} buses")

### Transformers

In [ ]:
dss.Circuit.SetActiveClass("transformer")
trafo_infos = json.loads(dss.ActiveClass.ToJSON())
for i, (trafo, trafo_info) in tqdm(enumerate(zip(dss.Transformers, trafo_infos))):
    name = trafo.Name()
    trafo_type = "delta" if trafo.IsDelta() else "wye"

    # Bus Connections
    bus_hv_name, *bus_hv_terminals = trafo_info["Bus"][0].split(".")
    #bus_hv_name = bus_hv_name.split("_")[0]
    bus_hv_id = net.bus.index[net.bus.name == bus_hv_name][0]
    bus_lv_name, *bus_lv_terminals = trafo_info["Bus"][1].split(".")
    #bus_lv_name = bus_lv_name.split("_")[0]
    bus_lv_id = net.bus.index[net.bus.name == bus_lv_name][0]

    # Voltage on each side
    vn_hv_kv, vn_lv_kv = trafo_info["kV"]
    # Rated Apparent Power
    sn_mva = min(trafo_info["kVA"]) / 1000.0

    # Tap-Changing Transformer Settings
    n_tap_steps = trafo.NumTaps()
    tap_settings = dict(
        min_tap_pu = trafo.MinTap(),  max_tap_pu = trafo.MaxTap(),
        tap_min = -n_tap_steps // 2, tap_max = n_tap_steps // 2,
        tap_step_percent = (trafo.MaxTap() - trafo.MinTap()) / n_tap_steps,
        tap_step_degree = 0,  tap_phase_shifter = False,
        tap_pos=0, tap_side = "hv",
    )
   
    # Place Holder (Mandatory Parameters)
    # Based on 20/0.4kV 0.25 MVA STD Type
    vkr_percent = 6.0 # Real Part Short-Circuit Voltage
    vk_percent = 1.44 # Relative Short-Circuit Voltage
    pfe_kw = 0.8 # Iron Losses (in kW)
    i0_percent = 0.32 # Open loop lossed in % of rated current
    shift_degree = 150
    
    pp.create_transformer_from_parameters(net, hv_bus=bus_hv_id, lv_bus=bus_lv_id, sn_mva=sn_mva,
                                          vn_hv_kv=vn_hv_kv, vn_lv_kv=vn_lv_kv,
                                          vkr_percent=vkr_percent, vk_percent = vk_percent,
                                          pfe_kw=pfe_kw, i0_percent=i0_percent, shift_degree=shift_degree,
                                          **tap_settings)

### Loads
Loads information on each load, does not currently load their profiles. 

In [ ]:
# Get loads as JSON (incl. their connected bus)
# TODO: Aggregate loads at same bus
dss.Circuit.SetActiveClass("load")
all_loads = json.loads(dss.ActiveClass.ToJSON())

phase_map = {"1":"a", "2":"b", "3":"c"}
for i, load in tqdm(enumerate(all_loads), total=len(all_loads)):
    name = load["Name"]
    n_phases = load["Phases"]
    bus_name, *bus_terminals = load["Bus1"].split(".")
    #bus_name = bus_name.split("_")[0]
    bus_id = net.bus.index[net.bus.name == bus_name][0]
    
    p_kW = load["kW"]
    pf = load["PF"]
    S_kva = (p_kW / pf)
    q_kvar = math.sqrt(math.pow(S_kva, 2)-math.pow(p_kW, 2))
    S_mva = S_kva / 1000.0

    # profile = load["Daily"]
    load_type = "wye" # TODO: Check this
    
    if n_phases == 1:
        # if np.any(np.isnan(net.asymmetric_load.sn_mva.values)):
        #     print(i, S_kva, np.sum(np.isnan(net.asymmetric_load.sn_mva.values)))
        phase = phase_map[bus_terminals[0]]
        if bus_id not in net.asymmetric_load.bus.values:
            pp.create_asymmetric_load(net, name=name, bus=bus_id, sn_mva=S_mva, type=load_type,
                        **{f"p_{phase}_mw":p_kW/1000.0, f"q_{phase}_mvar":q_kvar/1000.0})
        else: # Merge asymmetric loads at same bus
            mask = net.asymmetric_load.bus == bus_id
            if np.sum(mask) > 1:
                print(np.sum(mask))
                display(net.asymmetric_load[mask])
                raise ValueError("Mask matches multiple buses")
            net.asymmetric_load.loc[mask, [f"p_{phase}_mw", f"q_{phase}_mvar", "sn_mva"]] = \
            net.asymmetric_load.loc[mask, [f"p_{phase}_mw", f"q_{phase}_mvar", "sn_mva"]] + np.array([p_kW/1000.0, q_kvar/1000.0, S_mva])
    elif n_phases == 3: # Merge loads at same bus
        if bus_id not in net.load.bus.values:
            pp.create_load(net, name=name, bus=bus_id, sn_mva=S_mva, type=load_type,
                        p_mw=p_kW/1000.0, q_mvar=q_kvar/1000.0)
        else:
            mask = net.load.bus == bus_id
            if np.sum(mask) > 1: raise ValueError("Mask matches multiple buses")
            net.load.loc[mask, ["p_mw", "q_mvar", "sn_mva"]] = \
            net.load.loc[mask, ["p_mw", "q_mvar", "sn_mva"]] + np.array([p_kW/1000.0, q_kvar/1000.0, S_mva])
    else:
        raise ValueError(f"{n_phases} phase load not supported")

### Reactor

In [ ]:
for i, reactor in tqdm(enumerate(dss.Reactors), total=len(dss.Reactors.AllNames())):
    name = reactor.Name()

    bus1_name, *bus1_terminals = reactor.Bus1().split(".")
    #bus1_name = bus1_name.split("_")[0]
    bus1_id = net.bus.index[net.bus.name == bus1_name][0]
    bus2_name, *bus2_terminals = reactor.Bus2().split(".")
    #bus2_name = bus2_name.split("_")[0]
    bus2_id = net.bus.index[net.bus.name == bus2_name][0]
    
    r_ohm = reactor.R()
    x_ohm = reactor.X()
    r0_ohm, x0_ohm = reactor.Z0()
    if not reactor.IsDelta():
        pp.create_series_reactor_as_impedance(net, name=name,
                                            from_bus=bus1_id, to_bus=bus2_id,
                                            sn_mva=1.0, # Rated Power (Unknown) 
                                            r_ohm=r_ohm, x_ohm=x_ohm,
                                            r0_ohm=r0_ohm, x0_ohm=x0_ohm)

In [ ]:
import matplotlib.pyplot as plt
from pandapower.plotting import simple_plot

fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(32,24))
simple_plot(net, show_plot=False, ax=ax,
            bus_size=0.1, load_size=0.3, ext_grid_size=0.3, trafo_size=0.1, switch_size=0.4,
            plot_loads=True)
plt.tight_layout()
plt.savefig(Path.cwd() / "media" / "SpanishLVGrid.pdf")
plt.show()

In [ ]:
pp.drop_buses(net, buses=[0], drop_elements=True)
pp.create_continuous_bus_index(net)
pp.create_continuous_elements_index(net)
pp.to_json(net, grid_path / "grid.json")

In [ ]:
network = pp.from_json(grid_path / "grid.json")
simple_plot(network, plot_loads=True)